<a href="https://colab.research.google.com/github/786aafreen/Alibaba/blob/main/Data_Anal_Assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
import copy

In [ ]:
sns.set() #setting the default seaborn style for our plots

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Q.1 . For the given insurance dataset justify the hypothesis
1. Does bmi of males differ significantly from that of females?
2. Is the proportion of smokers significantly different in different genders?

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/insurance (1).csv') # read the data as a data frame

In [ ]:
df.head()

Basic EDA we have to here ::
1. First find the shape of the data,data type of individual columns.

2. Then check the null values.

3. Now Descriptive stats of numerical columns.

4. And then find out the distribution of numerical columns and the asssociated skewness and presence of outliers. 

5. Then need to find Distribution of categorical columns.

In [ ]:
df.info()

- The data has 1338 instances with 7 attributes, 2 integer type, 2 float type and 3 object type(Strings in the column).

#Check the null values

In [ ]:
df.isna().apply(pd.value_counts)

- There are no null values in any of the column


# five point summary of the continuous attributes

In [ ]:
df.describe().T

- Data looks legit as all the statistics seem reasonable
- Looking at the age column, data looks representative of the true age distribution of the adult population
- Very few people have more than 2 children. 75% of the people have 2 or less children
- The claimed amount is higly skewed as most people would require basic medi-care and only few suffer from diseases which cost more to get rid of

In [ ]:
#Plots to see the distribution of the continuous features individually

plt.figure(figsize= (20,15))
plt.subplot(3,3,1)
plt.hist(df.bmi, color='blue', edgecolor = 'black', alpha = 0.7)
plt.xlabel('bmi')

plt.subplot(3,3,2)
plt.hist(df.age, color='green', edgecolor = 'black', alpha = 0.7)
plt.xlabel('age')

plt.subplot(3,3,3)
plt.hist(df.charges, color='orange', edgecolor = 'black', alpha = 0.7)
plt.xlabel('charges')

plt.show()

- bmi looks quiet normally distributed.

- Age seems be be distributed quiet uniformly.

- As seen in the previous step, charges are highly skewed.

In [ ]:
Skewness = pd.DataFrame({'Skewness' : [stats.skew(df.bmi),stats.skew(df.age),stats.skew(df.charges)]},
                        index=['bmi','age','charges'])  # Measure the skeweness of the required columns
Skewness

- Skew of bmi is very less as seen in the previous step
- age is uniformly distributed and there's hardly any skew
- charges are highly skewed

In [ ]:
plt.figure(figsize= (20,15))
plt.subplot(3,1,1)
sns.boxplot(x= df.bmi, color='pink')

plt.subplot(3,1,2)
sns.boxplot(x= df.age, color='red')

plt.subplot(3,1,3)
sns.boxplot(x= df.charges, color='chocolate')

plt.show()

- bmi has a few extreme values
- charges as it is highly skewed, there are quiet a lot of extreme values

In [ ]:
plt.figure(figsize=(20,25))


x = df.smoker.value_counts().index    #Values for x-axis
y = [df['smoker'].value_counts()[i] for i in x]   # Count of each class on y-axis

plt.subplot(4,2,1)
plt.bar(x,y, align='center',color = 'red',edgecolor = 'black',alpha = 0.7)  #plot a bar chart
plt.xlabel('Smoker?')
plt.ylabel('Count ')

plt.title('Smoker distribution')

x1 = df.sex.value_counts().index    #Values for x-axis
y1 = [df['sex'].value_counts()[j] for j in x1]   # Count of each class on y-axis

plt.subplot(4,2,2)
plt.bar(x1,y1, align='center',color = 'gold',edgecolor = 'black',alpha = 0.7)  #plot a bar chart
plt.xlabel('Gender')

plt.ylabel('Count')
plt.title('Gender distribution')

x2 = df.region.value_counts().index    #Values for x-axis
y2 = [df['region'].value_counts()[k] for k in x2]   # Count of each class on y-axis

plt.subplot(4,2,3)
plt.bar(x2,y2, align='center',color = 'yellow',edgecolor = 'black',alpha = 0.7)  #plot a bar chart
plt.xlabel('Region')
plt.ylabel('Count ')
plt.title("Regions' distribution")

x3 = df.children.value_counts().index    #Values for x-axis
y3 = [df['children'].value_counts()[l] for l in x3]   # Count of each class on y-axis

plt.subplot(4,2,4)
plt.bar(x3,y3, align='center',color = 'lightblue',edgecolor = 'black',alpha = 0.7)  #plot a bar chart
plt.xlabel('No. of children')
plt.ylabel('Count ')
plt.title("Children distribution")

plt.show()

- There are a lot more non-smokers than there are smokers in the data
- Instances are distributed evenly accross all regions
- Gender is also distributed evenly
- Most instances have less than 2 children and very few have 4 or 5 children

#Bi-variate distribution of every possible attribute pair

In [ ]:
#Label encoding the variables before doing a pairplot because pairplot ignores strings
df_encoded = copy.deepcopy(df)
df_encoded.loc[:,['sex', 'smoker', 'region']] = df_encoded.loc[:,['sex', 'smoker', 'region']].apply(LabelEncoder().fit_transform) 

sns.pairplot(df_encoded)  #pairplot
plt.show()

- The only obvious correlation of 'charges' is with 'smoker'
- Looks like smokers claimed more money than non-smokers
- There's an interesting pattern between 'age' and 'charges. Could be because for the same ailment, older people are charged more than the younger ones

#Does bmi of males differ significantly from that of females?

In [ ]:
df.sex.value_counts()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(df.age, df.charges,hue=df.sex,palette= ['pink','lightblue'] )
plt.show()

- Visually, there is no apparent relation between gender and charges

In [ ]:
# T-test to check dependency of bmi on gender
Ho = "Gender has no effect on bmi"   # Stating the Null Hypothesis
Ha = "Gender has an effect on bmi"   # Stating the Alternate Hypothesis

x = np.array(df[df.sex == 'male'].bmi)  # Selecting bmi values corresponding to males as an array
y = np.array(df[df.sex == 'female'].bmi) # Selecting bmi values corresponding to females as an array

t, p_value  = stats.ttest_ind(x,y, axis = 0)  #Performing an Independent t-test

if p_value < 0.05:  # Setting our significance level at 5%
    print(f'{Ha} as the p_value ({p_value.round()}) < 0.05')
else:
    print(f'{Ho} as the p_value ({p_value.round(3)}) > 0.05')

#Gender has no effect on bmi as the p_value (0.09) > 0.05

#bmi of both the genders are identical

# Q.2 . Create portfolio for the given stocks:  &#39;GLD&#39;, &#39;AMZN
      for the last 10 years.
2. Visualize the expected returns on the 10 years series.
3. Evaluate the annual daily mean, correlation,   Sharpe ratio and daily standard mean.
4. . Discuss on the optimal portfolio and the different parameters evaluated for the portfolio.

The Sharpe ratio is the average return earned in excess of the risk-free rate per unit of volatility (in the stock market, volatility represents the risk of an asset)

In [ ]:
# Install yfinance package. 
!pip install yfinance 

In [ ]:
import yfinance as yf

#Create portfolio for the given stocks: GLD, amazon for the last 10 years.


Fetching the data from stocks of GLD and Amazon via specifying stock ticker

In [ ]:
GLD = yf.download('GLD','2012-12-12','2022-12-12')
AMZN =yf.download('AMZN','2012-12-12','2022-12-12')

Plot the Close Prices

In [ ]:
AMZN.Close.plot()
plt.show()

In [ ]:
GLD.Close.plot()
plt.show()

In [ ]:
GLD.tail()

In [ ]:
AMZN.drop(['Open','High','Low','Close','Volume'],axis=1,inplace=True)

In [ ]:
GLD.drop(['Open','High','Low','Close','Volume'],axis=1,inplace=True)

Now Normalize the Prices

In [ ]:
GLD.iloc[0]['Adj Close']

In [ ]:
for stock_df in(GLD,AMZN):
  stock_df['Normed Return']=stock_df['Adj Close']/stock_df.iloc[0]['Adj Close']

In [ ]:
AMZN.isnull().sum()

In [ ]:
GLD.isnull().sum()

Allocations
Now Let us profess that here are some allocations below for total portfolio:

* 65% in GLD
* 35% in Amazon

So these values must be reflected by multiplying our Normed Return via outer Allocations

In [ ]:
for stock_df,allo in zip([GLD,AMZN],[0.65,0.35]):
  stock_df['Allocation']=stock_df['Normed Return']*allo

Investment Let's profess we invested a million dollars in this portfolio

In [ ]:
for stock_df in (GLD,AMZN):
    stock_df['Position Values']=stock_df['Allocation']*1000000

In [ ]:
portfolio_val = pd.concat([GLD['Position Values'],AMZN['Position Values']],axis=1)

In [ ]:
portfolio_val.head()

In [ ]:
portfolio_val.columns = ['GLD_Pos','AMZN_Pos']

In [ ]:
portfolio_val.tail()

In [ ]:
portfolio_val['Total_Pos'] = portfolio_val.sum(axis=1)

In [ ]:
portfolio_val.tail()

Daily Return

In [ ]:
portfolio_val['Total_Pos'].pct_change(1)

In [ ]:
portfolio_val['Daily Returns']=portfolio_val['Total_Pos'].pct_change(1)
portfolio_val

Visualize the expected returns on the 10 years series.

In [ ]:
portfolio_val['Daily Returns'].plot(kind='kde')

In [ ]:
AMZN['Adj Close'].pct_change(1).plot(kind='kde')
GLD['Adj Close'].pct_change(1).plot(kind='kde')

Discuss on the optimal portfolio and the different parameters evaluated for the portfolio.(Assuming Greater Return of investment)

Let us assume our expected returns

In [ ]:
ExpectedReturn = [9.00,9.20,9.40,9.60,9.80,10.00,10.20,10.40,10.60,10.80,11.00]
StandardDeviation = [8.0,7.5,7.1,6.9,6.8,7.0,7.3,7.8,8.5,9.2,10.0]

In [ ]:
plt.scatter(StandardDeviation,ExpectedReturn,color='red')
plt.xlabel("StandardDeviation (in %)",color='Chocolate')
plt.ylabel("ExpectedReturn (in %)",color='brown')
plt.title("Markowitz Portfolio Analysis",color='Purple')
plt.show()

In [ ]:
portfolio_val['Total_Pos'].plot(figsize=(8,8),color='red')
plt.title('Total Portfolio Value')
plt.show()

In [ ]:
portfolio_val.drop('Total_Pos',axis=1).plot(kind='line')

Sharpe Ratio

In [ ]:
SR=portfolio_val['Daily Returns'].mean()/portfolio_val['Daily Returns'].std()
SR

In [ ]:
ASR=(252**0.5)*SR
ASR

In [ ]:
AMZN['Adj Close'].pct_change(1).plot(kind='kde')
GLD['Adj Close'].pct_change(1).plot(kind='kde')

correlation

In [ ]:
stock_data=pd.concat([GLD['Adj Close'],AMZN['Adj Close']],axis=1)

In [ ]:
stock_data.head()

In [ ]:
(stock_data/stock_data.iloc[0] * 100).plot(figsize = (10,8))

In [ ]:
logReturns = np.log(stock_data/stock_data.shift(1))
logReturns

Evaluate the annual daily mean, correlation, Sharpe ratio and daily standard mean.

In [ ]:
#To obtain annual average returns!
logReturns.mean() * 250

In [ ]:
#To obtain annual covariance between PG and Microsoft
logReturns.cov() * 250 

In [ ]:
stock_data.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(stock_data.corr())
plt.show()

In [ ]:
stocks = ['GLD', 'AMZN']

Dynamically generating weights code

In [ ]:
numberOfStocks = len(stocks)
numberOfStocks

In [ ]:
np.random.seed()
weights_1 = np.random.random(numberOfStocks)
weights_1

In [ ]:
weights = weights_1/np.sum(weights_1)
weights

Now Calculating expected return of portfolio

In [ ]:
(weights * logReturns.mean()).sum() * 250

Expected Standard Deviation and volatility

In [ ]:
np.sqrt(np.dot(weights.T,np.dot(logReturns.cov() * 250, weights)))

In [ ]:
expectedReturn = []
standardDeviation = []
weightList0 = []
weightList1 = []


# Running simulations for finding optimum weights
for i in range(120):
    weights = np.random.random(numberOfStocks)
    weights = weights/ weights.sum()
    weightList0.append(weights[0])
    weightList1.append(weights[1])
    
    expectedReturn.append((weights * logReturns.mean()).sum() * 250)
    standardDeviation.append(np.sqrt(np.dot(weights.T, np.dot(logReturns.cov() * 250, weights))))

#Converting lists into arrays

weightList1 = np.array(weightList1)  #Weights for AMZN
weightList1 = np.array(weightList1) #Weights for GLD
expectedReturn = np.array(expectedReturn) 
standardDeviation = np.array(standardDeviation)

#Creating dataframe
df = pd.DataFrame({"Weight of GLD": weightList0, "Weight of AMZN": weightList1, "Expected Return": expectedReturn, "Standard deviation": standardDeviation})
df.head()

In [ ]:
plt.figure(figsize=(14, 10), dpi=80)
plt.scatter(df["Standard deviation"], df["Expected Return"])
plt.xlabel("Standard deviation")
plt.ylabel("Expected return (in %)")
plt.show()

In [ ]:
df[(df["Expected Return"]>0.20) & (df["Expected Return"]< 0.23)].sort_values(by=['Expected Return'])

In [ ]:
df[(df["Expected Return"]>0.20)].sort_values(by=['Expected Return']).head(10)

In [ ]:
df["Expected Return"].mean()

In [ ]:
df["Expected Return"].sort_values().median()

In [ ]:
df[(df["Expected Return"]>0.25)].sort_values(by=['Expected Return'])

In [ ]:
df.loc[56]

In [ ]:
df.loc[21]

# Q.3 For the dataset that includes CO2 emissions from each energy resource starting January 1973
to July 2016. Answer the following

1. Check the stationary using ADF test and autocorrelation plot.
2. Forecast the target variable prediction using a suitable type of model.
3. Evaluate the different types of error residues to check fitness of good of the model.
4. Forecast the predictions for next 10 years on target variable.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 16

In [ ]:
import warnings
import itertools
warnings.filterwarnings("ignore") # specify to ignore warning messages

# Time series dataset retrieving and visualization

- First, in the following cells, we will retrive the monthly CO2 emissions dataset then we will visualize the dataset to decide the type of model we will use to model and analyse our time series (ts).

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/co2.csv")
df1 .head()

In [ ]:
df1.info()

The dataset has 6 columns where 2 of them are integer data type and 4 objects and 5094 observations.

#To read the dataset as a time series, we have to pass special arguments to the read_csv command as given below.

In [ ]:
dateparse = lambda x: pd.to_datetime(x, format='%Y%m', errors = 'coerce')
df1 = pd.read_csv("/content/drive/MyDrive/co2.csv", parse_dates=['YYYYMM'], index_col='YYYYMM', date_parser=dateparse) 
df1.head()

The arguments can be explained::

- parse_dates: This is a key to identify the date time column. Example, the column name is ‘YYYYMM’.
- index_col: This is a key that forces pandas to use the date time column as index.
- date_parser: Converts an input string into datetime variable.

In [ ]:
df1.tail(15)

In [ ]:
df1.Column_Order.value_counts()

# df1.Unit.value_counts()      Million Metric Tons of Carbon Dioxide    5094
#df1.Description.value_counts() values
#df1.MSN.value_counts() value

Total sum of CO2 emission from each energy group for every year is given as an observation that can be viewed in the NaT row. So, let us first identify and drop the non datetimeindex rows and also use ts to refere the time series dataset instead of the dataframe df. First, let us convert the index to datetime, coerce errors, and filter NaT

In [ ]:
ts = df1[pd.Series(pd.to_datetime(df1.index, errors='coerce')).notnull().values]
ts.head(15)

In [ ]:
ts.dtypes

As we can see from the ts data type, the emission value is represented as an object. Let us first convert the emision value into numeric value as follows

In [ ]:
#ss = ts.copy(deep=True)
ts['Value'] = pd.to_numeric(ts['Value'] , errors='coerce')
ts.head()

In [ ]:
ts.info()

4323 observations have emissions value and therefore, we need to drop the empty rows emissions value.

In [ ]:
ts.dropna(inplace = True)

 # Time series dataset visualization

The dataset has 8 energy sources of CO2 emission. In the following cell, we will group the CO2 Emission dataset based on the type of energy source.

In [ ]:
#group by products same products changing date(month)
Energy_sources = ts.groupby('Description')
Energy_sources.head()

The CO2 emission time series dataset is ploted to visualize the dependency of the emission in the power generation with time.

In [ ]:
CO2_per_source = ts.groupby('Description')['Value'].sum().sort_values()

In [ ]:
# I want to use shorter descriptions for the energy sources
CO2_per_source.index

In [ ]:
cols = ['Geothermal Energy', 'Non-Biomass Waste', 'Petroleum Coke','Distillate Fuel ',
        'Residual Fuel Oil', 'Petroleum', 'Natural Gas', 'Coal', 'Total Emissions']

In [ ]:
fig = plt.figure(figsize = (16,9))
x_label = cols
x_tick = np.arange(len(cols))
plt.bar(x_tick, CO2_per_source, align = 'center', alpha = 0.5)
fig.suptitle("CO2 Emissions by Electric Power Sector", fontsize= 25)
plt.xticks(x_tick, x_label, rotation = 70, fontsize = 20)
plt.yticks(fontsize = 20)
plt.xlabel('Carbon Emissions in MMT', fontsize = 20)
plt.show()

In [ ]:
df1.plot(figsize=(8,5))
plt.title('Monthly CO2 Concentration (ppm)')

plt.show()

From the bar chart, we can see that the contribution of coal to the total CO2 emission is significant followed by natural gas.

# Natural gas CO2 emission analysis

For developing the time series model and make forcasting, I will use the natural gas CO2 emission from the electirical power generetion. First, let us slice this data from the ts as follows:

In [ ]:
ts.head()

**Forecasting** is a technique that uses historical data as inputs to make informed estimates that are predictive in determining the direction of future trends. Businesses utilize forecasting to determine how to allocate their budgets or plan for anticipated expenses for an upcoming period of time.

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

In [ ]:
Emissions = ts.iloc[:,1:]

Monthly total emissions (mte)

In [ ]:
Emissions = ts.iloc[:,1:]   # Monthly total emissions (mte)
Emissions= Emissions.groupby(['Description', pd.TimeGrouper('M')])['Value'].sum().unstack(level = 0)
mte = Emissions['Natural Gas Electric Power Sector CO2 Emissions'] # monthly total emissions (mte)
mte.head()

In [ ]:
mte.tail()

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

In [ ]:
plt.plot(mte)

# Test stationary using Dickey-Fuller

In [ ]:
def TestStationaryPlot(ts):
    rol_mean = ts.rolling(window = 12, center = False).mean()
    rol_std = ts.rolling(window = 12, center = False).std()
    
    plt.plot(ts, color = 'blue',label = 'Original Data')
    plt.plot(rol_mean, color = 'red', label = 'Rolling Mean')
    plt.plot(rol_std, color ='black', label = 'Rolling Std')
    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    
    plt.xlabel('Time in Years', fontsize = 25)
    plt.ylabel('Total Emissions', fontsize = 25)
    plt.legend(loc='best', fontsize = 25)
    plt.title('Rolling Mean & Standard Deviation', fontsize = 25)
    plt.show(block= True)

In [ ]:
TestStationaryPlot(mte)

In [ ]:
def TestStationaryAdfuller(ts, cutoff = 0.01):

  ts_test = adfuller(ts, autolag = 'AIC')
  ts_test_output = pd.Series(ts_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
 
for key,value in ts_test[4].items():

  ts_test_output['Critical Value (%s)'%key] = value
  print(ts_test_output)
    
  if ts_test[1] <= cutoff:

    print("Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary")
  else:
    print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
TestStationaryAdfuller(mte)

Moving average

In this technique, we take average of ‘k’ consecutive values depending on the frequency of time series (in this case 12 monthes per year). Here, we will take the average over the past 1 year.

In [ ]:
moving_avg = mte.rolling(12).mean()
plt.plot(mte)
plt.plot(moving_avg, color='red')
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
plt.xlabel('Time (years)', fontsize = 25)
plt.ylabel('CO2 Emission (MMT)', fontsize = 25)
plt.title('CO2 emission from electric power generation', fontsize = 25)
plt.show()

In [ ]:
mte_moving_avg_diff = mte - moving_avg
mte_moving_avg_diff.head(13)

In [ ]:
mte_moving_avg_diff.dropna(inplace=True)
TestStationaryPlot(mte_moving_avg_diff)

In [ ]:
TestStationaryAdfuller(mte_moving_avg_diff)

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

The rolling mean values appear to be varying slightly. The Test Statistic is smaller than the 10% 5%, and 1% of critical values. So, we can say with 99% confidence level that the dataset is a stationary series.

# Exponentail weighted moving average

Another technique is to take the ‘weighted moving average’ where more recent values are given a higher weight. The popular method to assign the waights is using the exponential weighted moving average. Where weights are assigned to all previous values with a decay factor.

In [ ]:
mte_exp_wighted_avg = pd.ewma(mte, halflife=12)
plt.plot(mte)
plt.plot(mte_exp_wighted_avg, color='red')
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)
plt.xlabel('Time (years)', fontsize = 25)
plt.ylabel('CO2 Emission (MMT)', fontsize = 25)
plt.title('CO2 emission from electric power generation', fontsize = 25)
plt.show()

In [ ]:
mte_ewma_diff = mte - mte_exp_wighted_avg
TestStationaryPlot(mte_ewma_diff)

In [ ]:
TestStationaryAdfuller(mte_ewma_diff)

Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary 

This time series has lesser variations in mean and standard deviation compared to the orginal ddataset. Also, the Test Statistic is smaller than the 5% and 10% critical value, which is better than the original case. There will be no missing values as all values from starting are given weights. So, it will work even with no previous values. In this case, we can say with 95% confidence level the series is a stationary series.

# Eliminating trend and seasonality: Differencing

One of the most common method of dealing with both trend and seasonality is differencing. In this technique, we take the difference of the original observation at a particular instant with that at the previous instant. This mostly works well to improve stationarity. First order differencing can be done as follows:

In [ ]:
mte_first_difference = mte - mte.shift(1)  
TestStationaryPlot(mte_first_difference.dropna(inplace=False))

In [ ]:
TestStationaryAdfuller(mte_first_difference.dropna(inplace=False))

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

The first difference improves the stationarity of the series significantly. Let us use also the seasonal difference to remove the seasonality of the data and see how that impacts stationarity of the data.

In [ ]:
mte_seasonal_difference = mte - mte.shift(12)  
TestStationaryPlot(mte_seasonal_difference.dropna(inplace=False))
TestStationaryAdfuller(mte_seasonal_difference.dropna(inplace=False))

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

Compared to the original data the seasonal difference also improves the stationarity of the series. The next step is to take the first difference of the seasonal difference.

In [ ]:
mte_seasonal_first_difference = mte_first_difference - mte_first_difference.shift(12)  
TestStationaryPlot(mte_seasonal_first_difference.dropna(inplace=False))

In [ ]:
TestStationaryAdfuller(mte_seasonal_first_difference.dropna(inplace=False))

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

Now, if we look the Test Statistic and the p-value, taking the seasonal first difference has made our the time series dataset stationary. This differencing procedure could be repeated for the log values, but it didn’t make the dataset any more stationary.

In [ ]:
mte_seasonal_first_difference = mte_first_difference - mte_first_difference.shift(12)  
TestStationaryPlot(mte_seasonal_first_difference.dropna(inplace=False))

In [ ]:
TestStationaryAdfuller(mte_seasonal_first_difference.dropna(inplace=False))

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

Now, if we look the Test Statistic and the p-value, taking the seasonal first difference has made our the time series dataset stationary. This differencing procedure could be repeated for the log values, but it didn’t make the dataset any more stationary.

# Eliminating trend and seasonality: Decomposing

In this technique, it statrating by modeling both trend and seasonality and removing them from the model.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(mte)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(mte, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

Here we can see that the trend, seasonality are separated out from data and we can model the residuals. Lets check stationarity of residuals:

In [ ]:
mte_decompose = residual
mte_decompose.dropna(inplace=True)
TestStationaryPlot(mte_decompose)
TestStationaryAdfuller(mte_decompose)

Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary

**Find optimal parameters and build SARIMA model**

When looking to fit time series dataset with seasonal **ARIMA** model, our first goal is to find the values of **SARIMA**(p,d,q)(P,D,Q)s that optimize our metric of interest. Before moving directly how to find the optimal values of the parameters let us see the two situations in stationarities:

This is the easy case wherein we can model the residuals as **white noise**. The second case being a series with significant dependence among values and needs statistical models like** ARIMA to forecast future oucomes.**

**Auto-Regressive Integrated Moving Average (ARIMA):** The ARIMA forecasting for a stationary time series is a linear funcion similar to linear regression. The predictors mainly depend on the parameters (p,d,q) of the ARIMA model:

#Plot the ACF and PACF charts and find the optimal parameters

**- Autocorrelation Function (ACF)**: It is a measure of the correlation between the the time series (ts) with a lagged version of itself. For instance at lag 4, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-4’…’t2-4’ (t1-4 and t2 being end points of the range).


**- Partial Autocorrelation Function (PACF):** This measures the correlation between the ts with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 4, it will check the correlation but remove the effects already explained by lags 1 to 3.

Therefore, the next step will be determing the **tuning parameters** (p and q) of the model by looking at the a**utocorrelation and partial autocorrelation graphs**. The chart below provides a brief guide on how to read the autocorrelation and partial autocorrelation graphs inorder to select the parameters.

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(mte_seasonal_first_difference.iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(mte_seasonal_first_difference.iloc[13:], lags=40, ax=ax2)

**Grid search**

To find the optimal parameters for **ARIMA models** using the **graphical method is not trivial** and it is time consuming. We will select the optimal parameter values systematically using the grid search (**hyperparameter optimization**) method. The grid search iteratively explore different combinations of the parameters. For each combination of parameters, we will fit a new seasonal ARIMA model with the **SARIMAX()** function from the statsmodels module and assess its overall quality. Once we have explored the entire landscape of parameters, our optimal set of parameters will be the one that yields the best performance for our criteria of interest. Let's begin by generating the various combination of parameters that we wish to assess:

In [ ]:
p = d = q = range(0, 2) # Define the p, d and q parameters to take any value between 0 and 2
pdq = list(itertools.product(p, d, q)) # Generate all different combinations of p, q and q triplets
pdq_x_QDQs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))] # Generate all different combinations of seasonal p, q and q triplets
print('Examples of Seasonal ARIMA parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], pdq_x_QDQs[1]))
print('SARIMAX: {} x {}'.format(pdq[2], pdq_x_QDQs[2]))

In [ ]:
print(pdq)
print(pdq_x_QDQs)

When evaluating and comparing statistical models fitted with different parameters, each can be ranked against one another based on how well it fits the data or its ability to accurately predict future data points. We will use the **AIC** (Akaike Information Criterion) value, which is conveniently returned with **ARIMA models** fitted using **statsmodels**. The **AIC** **measures how well a model fits the data** while taking into account the overall complexity of the model. A model that fits the data very well while using lots of features will be assigned a **larger AIC score than a model** that uses fewer features to achieve the same **goodness-of-fit**. The lowest AIC refore, we are interested in finding the model that yields the lowest AIC value.

The order argument specifies the (p, d, q) parameters, while the seasonal_order argument specifies the (P, D, Q, S) seasonal component of the Seasonal ARIMA model. After fitting each SARIMAX()model, the code prints out its respective AIC score.

Notes on AIC score: AIC will choose the best model from a set(**The “best” model will be the one that neither under-fits nor over-fits**.),then consider running a hypothesis test to figure out the relationship between the variables in your model and the outcome of interest.

In [ ]:
for param in pdq:
    for seasonal_param in pdq_x_QDQs:
        try:
            mod = sm.tsa.statespace.SARIMAX(mte,
                                            order=param,
                                            seasonal_order=seasonal_param,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
          continue

In [ ]:
a=[]
b=[]
c=[]
wf=pd.DataFrame()

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in pdq_x_QDQs:
        try:
            mod = sm.tsa.statespace.SARIMAX(mte,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            a.append(param)
            b.append(param_seasonal)
            c.append(results.aic)
        except:
            continue
wf['pdq']=a
wf['pdq_x_QDQs']=b
wf['aic']=c
print(wf[wf['aic']==wf['aic'].min()])

SARIMAX(1, 1, 1)x(0, 1, 1, 12) yields the lowest AIC value of 2003.553. Therefore, we will consider this to be optimal option out of all the parameter combinations.W e have identified the set of parameters that produces the best fitting model to our time series data. We can proceed to analyze this particular model in more depth

In [ ]:
mod = sm.tsa.statespace.SARIMAX(mte, 
                                order=(1,1,1), 
                                seasonal_order=(0,1,1,12),   
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary())

The coef column shows the weight (i.e. importance) of each feature and how each one impacts the time series. The P>|z| column informs us of the significance of each feature weight. Here, each weight has a p-value close to 0, so it is reasonable to include the features in our model.

When fitting seasonal ARIMA models, it is important to run model diagnostics to ensure that none of the assumptions made by the model have been violated. First, we get a line plot of the residual errors, suggesting that there may still be some trend information not captured by the model.

In [ ]:
results.resid.plot(figsize=(12,8))

In [ ]:
print(results.resid.describe())

The figure displays the distribution of the residual errors. It shows a little bias in the prediction. Next, we get a density plot of the residual error values, suggesting the errors are Gaussian, but may not be centered on zero.

In [ ]:
results.resid.plot(figsize=(12,8),kind='kde')

The plot_diagnostics object allows us to quickly generate model diagnostics and investigate for any unusual behavior.

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

Our primary concern is to ensure that the residuals of our model are uncorrelated and normally distributed with zero-mean. If the seasonal ARIMA model does not satisfy these properties, it is a good indication that it can be further improved.

The model diagnostic suggests that the model residual is normally distributed based on the following::

- In the top right plot, the red KDE line follows closely with the N(0,1) line. Where, N(0,1) is the standard notation for a normal distribution with mean 0 and standard deviation of 1. 

- This is a good indication that the residuals are normally distributed. The forecast errors deviate somewhat from the straight line, indicating that the normal distribution is not a perfect model for the distribution of forecast errors, but it is not unreasonable.

- The qq-plot on the bottom left shows that the ordered distribution of residuals (blue dots) follows the linear trend of the samples taken from a standard normal distribution. Again, this is a strong indication that the residuals are normally distributed.

- The residuals over time (top left plot) don't display any obvious seasonality and appear to be white noise. This is confirmed by the autocorrelation (i.e. correlogram) plot on the bottom right,which shows that the time series residuals have low correlation with lagged versions of itself.

- Those observations lead us to conclude that our model produces a satisfactory fit that could help us understand our time series data and forecast future values.

**Validating prediction**

We have obtained a model for our time series that can now be used to produce **forecasts**. We start by comparing predicted values to real values of the time series, which will help us understand the accuracy of our forecast. The get_prediction() and conf_int() attributes allow us to obtain the values and associated confidence intervals for **forecasts of the time series.**

In [ ]:
pred = results.get_prediction(start = 480, end = 523, dynamic=False)
pred_ci = pred.conf_int()
pred_ci.head()

The dynamic=False argument ensures that we produce one-step ahead forecasts, meaning that forecasts at each point are generated using the full history up to that point.

We can plot the real and forecasted values of the CO2 emission time series to assess how well the model fits.

In [ ]:
ax = mte['1973':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='r', alpha=.5)

ax.set_xlabel('Time (years)')
ax.set_ylabel('NG CO2 Emissions')

plt.legend()

plt.show()

Overall, our forecasts align with the true values very well, showing an overall similar behavior.

It is also useful to quantify the accuracy of our forecasts. We will use the MSE (Mean Squared Error), which summarizes the average error of our forecasts. For each predicted value, we compute its distance to the true value and square the result. The results need to be squared so that positive/negative differences do not cancel each other out.

In [ ]:
mte_forecast = pred.predicted_mean
mte_truth = mte['2013-01-31':]

# Compute the mean square error
mse = ((mte_forecast - mte_truth) ** 2).mean()
print('The Mean Squared Error (MSE) of the forecast is {}'.format(round(mse, 2)))
print('The Root Mean Square Error (RMSE) of the forcast: {:.4f}'
      .format(np.sqrt(sum((mte_forecast-mte_truth)**2)/len(mte_forecast))))

The Mean Squared Error (MSE) of the forecast is 4.09
The Root Mean Square Error (RMSE) of the forcast: nan

In [ ]:
mte_pred_concat = pd.concat([mte_truth, mte_forecast])

The goal of developing the model is to get a good quality predictive power using dynamic forecast. That is, we use information from the time series up to a certain point, and after that, forecasts are generated using values from previous forecasted time points as follows:

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2013-01-31'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

From, plotting the observed and forecasted values of the time series, we see that the overall forecasts are accurate even when we use the dynamic forecast. All forecasted values (red line) match closely to the orginal observed (blue line) data, and are well within the confidence intervals of our forecast.

In [ ]:
ax = mte['1973':].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], 
                color='r', 
                alpha=.3)

ax.fill_betweenx(ax.get_ylim(), 
                 pd.to_datetime('2013-01-31'), 
                 mte.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Time (years)')
ax.set_ylabel('CO2 Emissions')

plt.legend()
plt.show()

In [ ]:
# Extract the predicted and true values of our time series
mte_forecast = pred_dynamic.predicted_mean
mte_orginal = mte['2013-01-31':]

# Compute the mean square error
mse = ((mte_forecast - mte_orginal) ** 2).mean()
print('The Mean Squared Error (MSE) of the forecast is {}'.format(round(mse, 2)))
print('The Root Mean Square Error (RMSE) of the forcast: {:.4f}'
      .format(np.sqrt(sum((mte_forecast-mte_orginal)**2)/len(mte_forecast))))

**Forecasting**

In [ ]:
# Get forecast of 10 years or 120 months steps ahead in future
forecast = results.get_forecast(steps= 120)
# Get confidence intervals of forecasts
forecast_ci = forecast.conf_int()
forecast_ci.head()

In [ ]:
ax = mte.plot(label='observed', figsize=(20, 15))
forecast.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(forecast_ci.index,
                forecast_ci.iloc[:, 0],
                forecast_ci.iloc[:, 1], color='g', alpha=.4)
ax.set_xlabel('Time (year)')
ax.set_ylabel('NG CO2 Emission level')

plt.legend()
plt.show()

Both the forecast and associated confidence interval that we have generated can now be used to further explore and understand the time series. The forecast shows that the CO2 emission from natural gas power generation is expected to continue increasing.

**Conclusion**

In this notebook, I have explored how to retrieve CSV dataset, how to transform the dataset into times series, testing if the time series is stationary or not using graphical and Dickey-Fuller test statistic methods, how to transform time series to stationary, how to find optimal parameters to build SARIMA model using grid search method, diagnosing time series prediction, validating the predictive power, forecasting 10 year future CO2 emission from power generation using natural gas.

Future work: developing a time series model of natural gas forecasing